In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from IPython.display import display
import os
from google.colab import drive

In [3]:
# Mount google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# Flower Folder Path
flower_images = '/content/drive/MyDrive/5_flower_classifier'

In [13]:
# Train set transformations
train_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.3),
    transforms.ColorJitter(brightness=0.5,contrast=0.5,saturation=0.5,hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=3,sigma=(0.1,2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


# Validation set transformations

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),

])

#The code below was used to create a train and test set from my flower images.

In [10]:
import os
import shutil
import random

# Path to original dataset
src_folder = "/content/drive/MyDrive/5_flower_classifier/flowers"
# Paths to output train/test folders
train_folder = "/content/drive/MyDrive/5_flower_classifier/train"
test_folder = "/content/drive/MyDrive/5_flower_classifier/test"

# Create train/test folders
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Set train/test split ratio
train_ratio = 0.8  # 80% train, 20% test

# Iterate through each class folder
for class_name in os.listdir(src_folder):
    class_path = os.path.join(src_folder, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)  # shuffle images

    split_index = int(len(images) * train_ratio)
    train_images = images[:split_index]
    test_images = images[split_index:]

    # Create class folders in train/test directories
    os.makedirs(os.path.join(train_folder, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_folder, class_name), exist_ok=True)

    # Move/copy images
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_folder, class_name, img))
    for img in test_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_folder, class_name, img))

print("Dataset split completed!")


Dataset split completed!


In [14]:
# Create the datasets for train and test
train_dataset = datasets.ImageFolder("/content/drive/MyDrive/5_flower_classifier/train", transform=train_transform)
test_dataset = datasets.ImageFolder("/content/drive/MyDrive/5_flower_classifier/test", transform=test_transform)

In [15]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [16]:
# Classes
print(train_dataset.classes)

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


# Defining Model / Training

In [14]:
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 115MB/s]


In [16]:
# number of input features
num_features = model.fc.in_features
num_features

2048

In [17]:
model.fc = nn.Sequential(
    nn.Linear(num_features,512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512,5)
)



In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [6]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
def train(model,dataloader,optimizer,epochs):
  model.to(device)

  for epoch in range(epochs):
    model.train()
    running_loss=0.0
    for b, (X,y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      optimizer.zero_grad()
      y_pred = model(X)
      loss = criterion(y_pred,y)

      loss.backward()
      optimizer.step()

      running_loss += loss.item()

      if (b + 1) % 10 == 0:  # every 10 batches
        print(f"Epoch [{epoch+1}/{epochs}], Batch [{b+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}")

    torch.save(model.state_dict(), f"/content/drive/MyDrive/5_flower_classifier/checkpoint_epoch{epoch+1}.pth")

In [21]:
# Freeze layers
for param in model.parameters():
  param.requires_grad = False

for param in model.fc.parameters():
  param.requires_grad = True

In [22]:
# Train the model

train(model,train_loader,10)

Epoch [1/10], Batch [10/108], Loss: 1.4902
Epoch [1/10], Batch [20/108], Loss: 1.4364
Epoch [1/10], Batch [30/108], Loss: 1.4419
Epoch [1/10], Batch [40/108], Loss: 1.3703
Epoch [1/10], Batch [50/108], Loss: 1.3034
Epoch [1/10], Batch [60/108], Loss: 1.3282
Epoch [1/10], Batch [70/108], Loss: 1.2384
Epoch [1/10], Batch [80/108], Loss: 1.0835
Epoch [1/10], Batch [90/108], Loss: 1.0814
Epoch [1/10], Batch [100/108], Loss: 1.0015
Epoch 1/10, Loss: 1.2851
Epoch [2/10], Batch [10/108], Loss: 0.9819
Epoch [2/10], Batch [20/108], Loss: 0.9627
Epoch [2/10], Batch [30/108], Loss: 0.9339
Epoch [2/10], Batch [40/108], Loss: 0.9072
Epoch [2/10], Batch [50/108], Loss: 0.7377
Epoch [2/10], Batch [60/108], Loss: 1.0210
Epoch [2/10], Batch [70/108], Loss: 0.8086
Epoch [2/10], Batch [80/108], Loss: 0.7543
Epoch [2/10], Batch [90/108], Loss: 0.7076
Epoch [2/10], Batch [100/108], Loss: 0.9515
Epoch 2/10, Loss: 0.9426
Epoch [3/10], Batch [10/108], Loss: 0.8406
Epoch [3/10], Batch [20/108], Loss: 0.7380
Ep

In [23]:
torch.save(model.state_dict(), "/content/drive/MyDrive/5_flower_classifier/first_model.pth")

In [7]:
current_model = models.resnet50(pretrained=False)
num_features = current_model.fc.in_features
current_model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 5)
)

current_model.load_state_dict(torch.load("/content/drive/MyDrive/5_flower_classifier/first_model.pth"))
current_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
#Bring the flower labels in
flowers = train_dataset.classes
flowers

NameError: name 'train_dataset' is not defined

In [20]:
def evaluate(model,loader,criterion,device):
  model.eval()
  running_loss = 0.0
  correct = 0
  total = 0
  max_batches = 20

  with torch.no_grad():
    for i, (X, y) in enumerate(loader):

      X,y = X.to(device), y.to(device)

      outputs = model(X)
      loss = criterion(outputs,y)
      running_loss += loss.item()

      #predictions
      _, preds = torch.max(outputs,1)
      correct += (preds == y).sum().item()
      total += y.size(0)

  avg_loss = running_loss / len(loader)
  accuracy = 100 * correct / total
  print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
  return avg_loss, accuracy


In [29]:
evaluate(current_model, test_loader, criterion,device)

Validation Loss: 0.4065, Accuracy: 84.16%


(0.4064834235302572, 84.16184971098266)

In [38]:
def predict(model,X,device,class_name):
  with torch.no_grad():

    model.eval()
    output = model(X)
    _,preds = torch.max(output,1)
    return flowers[preds]



In [41]:
image_path = "/content/drive/MyDrive/5_flower_classifier/sunflower3.jpg"
image = Image.open(image_path).convert("RGB")

# transform setting to image
t_img = test_transform(image)
# Add batch dimensions
test_img = t_img.unsqueeze(0).to(device)

pred_name = predict(current_model, test_img, device, flowers)

print(f"Predicted: {pred_name}")

Predicted: sunflower


In [8]:
# Perform further training to see if we can get it a little more accurate
criterion = nn.CrossEntropyLoss()
optimizer2 = torch.optim.Adam([
    {'params': current_model.layer4.parameters(), 'lr': 1e-4},
    {'params': current_model.fc.parameters(), 'lr': 1e-3}
])

for param in current_model.parameters():
    param.requires_grad = False

for param in current_model.layer4.parameters():  # unfreeze last block
    param.requires_grad = True

for param in current_model.fc.parameters():  # ensure classifier is trainable
    param.requires_grad = True

In [19]:
train(current_model,train_loader,optimizer2,5)

Epoch [1/5], Batch [10/108], Loss: 0.9644
Epoch [1/5], Batch [20/108], Loss: 0.5550
Epoch [1/5], Batch [30/108], Loss: 0.9210
Epoch [1/5], Batch [40/108], Loss: 0.6496
Epoch [1/5], Batch [50/108], Loss: 0.6486
Epoch [1/5], Batch [60/108], Loss: 0.5866
Epoch [1/5], Batch [70/108], Loss: 0.5689
Epoch [1/5], Batch [80/108], Loss: 0.7999
Epoch [1/5], Batch [90/108], Loss: 0.4287
Epoch [1/5], Batch [100/108], Loss: 0.3820
Epoch 1/5, Loss: 0.7003
Epoch [2/5], Batch [10/108], Loss: 0.2586
Epoch [2/5], Batch [20/108], Loss: 0.6055
Epoch [2/5], Batch [30/108], Loss: 0.6458
Epoch [2/5], Batch [40/108], Loss: 0.9930
Epoch [2/5], Batch [50/108], Loss: 0.4249
Epoch [2/5], Batch [60/108], Loss: 0.4411
Epoch [2/5], Batch [70/108], Loss: 0.7283
Epoch [2/5], Batch [80/108], Loss: 0.5875
Epoch [2/5], Batch [90/108], Loss: 0.6342
Epoch [2/5], Batch [100/108], Loss: 0.3410
Epoch 2/5, Loss: 0.5766
Epoch [3/5], Batch [10/108], Loss: 0.4590
Epoch [3/5], Batch [20/108], Loss: 0.4996
Epoch [3/5], Batch [30/108

In [21]:
evaluate(current_model, test_loader, criterion,device)

Validation Loss: 0.2205, Accuracy: 90.98%


(0.22049897097534604, 90.98265895953757)

In [22]:
#Best model so far
torch.save(current_model.state_dict(), "/content/drive/MyDrive/5_flower_classifier/best_model.pth")